# Modelo AWD-LSTM

In [ ]:
import os
os.getcwd()

In [ ]:
from fastai.text.all import *
from IPython.display import display, HTML

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
path = Path('/kaggle/working')
model_path = Path('models')

## Regresión con biografías

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_reg = pd.read_csv('../input/es-wiki/Etiquetado_Mano1.csv')
df_reg[['F', 'D/C', 'A/C/F', 'L', 'P', 'Ac']] = df_reg[['F', 'D/C', 'A/C/F', 'L', 'P', 'Ac']].apply(pd.to_numeric)

In [ ]:
train_df, test_df = train_test_split(df_reg, test_size=0.3, random_state=21)
df = train_df
df.head()

In [ ]:
train_dls = TextDataLoaders.from_df(df=train_df, valid_pct=0.1, seed=42, text_col='bio', text_vocab=dls_finanzas.vocab, bs=64, seq_len=512,
                                  label_col=['F', 'D/C', 'A/C/F', 'L', 'P', 'Ac'],
                                  y_block=RegressionBlock(n_out=6))

In [ ]:
learn = text_classifier_learner(train_dls, AWD_LSTM, drop_mult=0.5,
                                n_out=6,
                                y_range=(0, 1), metrics=[rmse]
                                ).to_fp16()

In [ ]:
learn.load_encoder('es_wiki_fin_encoder')

In [ ]:
learn.fit_one_cycle(1, 1e-2)
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)
learn.freeze_to(-3)
learn.fit_one_cycle(1, 1e-2)
learn.unfreeze()
learn.fit_one_cycle(3, 1e-2)

In [ ]:
learn.fit_one_cycle(250, 1e-2)

In [ ]:
learn.save('es_wiki_reg')